In [29]:
import sys
sys.path.append("../script/")

import Functions
from Engine import Engine
from GTGP import GTGP


from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import importlib
from time import time
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
import pandas as pd
from time import time

In [30]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """
    
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    print(roc_auc_score(true_y,prob[:,1]))

# data

In [31]:
from sklearn.datasets import load_digits
mnist = load_digits()

X = pd.DataFrame(mnist.data).values
y = pd.DataFrame(mnist.target).values.flatten()

seeds = [10086, 200,500,30506,30405,30420,10056,7059,40965,5398,869543,83491,823190,
         48392,2810,48392,3498210,483902,859032,12890,538920,86954,54309,6504,9840,
         219805,548,2981,432890,5438908,219094,5843902,60854,979,12890,2108,4093]
train_size = 10

In [32]:
dataset = 'mnist_8_8_10'

# model

In [33]:
def fit_trees():
    learning_rate=0.3
    max_depth=1
    bins=8
    lam=1

    gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

    total_size=10
    elite_size = 10
    epoch=1000
    gp_epoch=3
    verbose = 0
    tolerance=0.001

    gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

    retrain_epoch=10
    alpha=0
    beta=1
    gammer=0

    verbose=1
    gtgp.lam = 1
    gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

    return gtgp

In [34]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    gtgp = fit_trees()

    print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
    print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

    num_trees,depth,num_nodes = gtgp.print_model()

    train_acc = accuracy_score(y_train,np.argmax(gtgp.train_p,axis=1))
    test_acc = accuracy_score(y_test,np.argmax(gtgp.test_p,axis=1))
    train_f1 = f1_score(y_train,np.argmax(gtgp.train_p,axis=1),average='macro')
    test_f1 = f1_score(y_test,np.argmax(gtgp.test_p,axis=1),average='macro')


    with open('./benchmark/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

retrain  1 :
	train: 1.0 1.0607179607058819 	test: 0.540570789031897 1253.3150437810327
retrain  2 :
	train: 1.0 0.285501431333214 	test: 0.5400111919418019 1148.607521718053
retrain  3 :
	train: 1.0 0.12512195637132356 	test: 0.5400111919418019 1109.2053100927226
retrain  4 :
	train: 1.0 0.06791384446137747 	test: 0.5416899832120873 1091.9999309610794
retrain  5 :
	train: 1.0 0.03600852861455803 	test: 0.5411303861219922 1080.9300824976335
retrain  6 :
	train: 1.0 0.02201695256912526 	test: 0.5422495803021824 1074.279057821641
retrain  7 :
	train: 1.0 0.013383923050321352 	test: 0.5422495803021824 1070.0918206415045
retrain  8 :
	train: 1.0 0.009676648135376558 	test: 0.5416899832120873 1070.3549128868908
retrain  9 :
	train: 1.0 0.007777559834740929 	test: 0.5422495803021824 1068.6986078034913
retrain  10 :
	train: 1.0 0.006133533889250023 	test: 0.5416899832120873 1067.5710500603248
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000   

In [15]:
def find_terminals(node):
    if node.terminal == True:
        return [node.index]
    else:
        ts = []
        for n in node.sons:
            ts.extend(find_terminals(n))
        return ts


terminals = []
for node in gtgp.stack:
    terminals.extend(find_terminals(node))
counts = pd.value_counts(terminals)


# div = len(counts)//1
div = 20
features = counts[:div].keys().tolist()

X_train = X_train[:,features]
X_test = X_test[:,features]

counts,len(np.unique(terminals))

(26    54
 42    50
 36    46
 54    45
 13    44
       ..
 39     4
 32     3
 56     2
 0      2
 24     2
 Name: count, Length: 64, dtype: int64,
 64)

In [352]:
gtgp.print_model()

----------------GTGP-------------
Number of Trees: 1681
Number of nodes: 5521


In [333]:
np.average([tree.depth for tree in gtgp.stack])

1.1508670520231215

In [334]:
print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        99
           1    1.00000   1.00000   1.00000       101
           2    1.00000   1.00000   1.00000        98
           3    1.00000   1.00000   1.00000       102
           4    1.00000   1.00000   1.00000       101
           5    1.00000   1.00000   1.00000       101
           6    1.00000   1.00000   1.00000       101
           7    1.00000   1.00000   1.00000       100
           8    1.00000   1.00000   1.00000        97
           9    1.00000   1.00000   1.00000       100

    accuracy                        1.00000      1000
   macro avg    1.00000   1.00000   1.00000      1000
weighted avg    1.00000   1.00000   1.00000      1000

              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        79
           1    0.98780   1.00000   0.99387        81
           2    0.97531   1.00000   0.98750        79
           3    0.98750 

# Decision Tree

In [35]:
from sklearn.tree import DecisionTreeClassifier

In [36]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    clf = DecisionTreeClassifier()
    clf.fit(X_train,y_train)

    num_trees = 1
    depth = clf.tree_.max_depth
    num_nodes = clf.tree_.node_count

    train_acc = accuracy_score(y_train,clf.predict(X_train))
    test_acc = accuracy_score(y_test,clf.predict(X_test))
    train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

    with open('./benchmark_DC/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# XGBoost

In [9]:
import xgboost


In [37]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=4,n_estimators=100)
    # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=4)
    # xgb = xgboost.XGBClassifier()
    xgb.fit(X_train,y_train)

    import json

    def item_generator(json_input, lookup_key):
        if isinstance(json_input, dict):
            for k, v in json_input.items():
                if k == lookup_key:
                    yield v
                else:
                    yield from item_generator(v, lookup_key)
        elif isinstance(json_input, list):
            for item in json_input:
                yield from item_generator(item, lookup_key)

    def tree_depth(json_text):
        json_input = json.loads(json_text)
        depths = list(item_generator(json_input, 'depth'))
        return max(depths) + 1 if len(depths) != 0 else 1

    train_acc = accuracy_score(y_train,xgb.predict(X_train))
    test_acc = accuracy_score(y_test,xgb.predict(X_test))
    train_f1 = f1_score(y_train,xgb.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,xgb.predict(X_test),average='macro')

    booster = xgb.get_booster()

    tree_df = booster.trees_to_dataframe()
    depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
    num_trees = len(depths)
    depth = np.average(depths)
    num_nodes = len(tree_df)

    with open('./benchmark_xgb/mnist_8_8_10.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# GDBT

In [38]:
from sklearn.ensemble import GradientBoostingClassifier

In [39]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    clf = GradientBoostingClassifier()
    # clf = GradientBoostingClassifier()
    clf.fit(X_train,y_train)

    train_acc = accuracy_score(y_train,clf.predict(X_train))
    test_acc = accuracy_score(y_test,clf.predict(X_test))
    train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

    num_trees = len([ est for ests in clf.estimators_ for est in ests])
    depth = np.average([ max(1,est.tree_.max_depth) for ests in clf.estimators_ for est in ests])
    num_nodes = sum([ est.tree_.node_count for ests in clf.estimators_ for est in ests])

    with open('./benchmark_GDBT/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# Random Forest

In [40]:
from sklearn.ensemble import RandomForestClassifier

In [41]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    rfc = RandomForestClassifier()
    rfc.fit(X_train,y_train)

    train_acc = accuracy_score(y_train,rfc.predict(X_train))
    test_acc = accuracy_score(y_test,rfc.predict(X_test))
    train_f1 = f1_score(y_train,rfc.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,rfc.predict(X_test),average='macro')

    num_trees = len(rfc.estimators_)
    depth = np.average([est.tree_.max_depth for est in rfc.estimators_])
    num_nodes = sum([est.tree_.node_count for est in rfc.estimators_])

    with open('./benchmark_RF/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)